# data.preprocess

> Fill in a module description here

In [ ]:
#| default_exp data.preprocess

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt

In [ ]:
#| export
def weekday_average(data:list[float])->list[float]:
    weekday_avgs = data.groupby(data.index.weekday).mean() * 24
    return weekday_avgs.values.flatten()

In [ ]:
#| export
def day_segment_average(data:list[float])->list[float]:
    segments = {
        'early_morning': (7, 9),
        'morning': (9, 13),
        'early_afternoon': (13, 17),
        'late_afternoon': (17, 21),
        'night': [(21, 24), (0, 7)]
    }
    averages = []
    for segment, hours in segments.items():
        if segment == 'night':
            energy = data.between_time('21:00', '23:59').mean() * 10 + data.between_time('00:00', '06:59').mean() * 10
        elif  segment == 'early_morning':
            start, end = hours
            energy = data.between_time(f'{start}:00', f'{end-1}:59').mean() * 2
        else :
            start, end = hours
            energy = data.between_time(f'{start}:00', f'{end-1}:59').mean() * 4
            
        averages.append(energy)
    return np.array(averages).flatten()

In [ ]:
#| export
def total_energy_used(data:list[float])->float:
    return data.sum()

In [ ]:
#| export
def average_energy_used(data:list[float])->list[float]:
    hourly_avg = data.mean()
    daily_avg = data.resample('D').sum().mean()
    weekly_avg = data.resample('W').sum().mean()
    monthly_avg = data.resample('ME').sum().mean()
    return np.array([hourly_avg, daily_avg, weekly_avg, monthly_avg]).flatten()

In [ ]:
#| export
def weekend_businessday_avg(data:list[float])->list[float]:
    weekends_avg = data[data.index.weekday >= 5].resample('D').sum().mean() 
    business_days_avg = data[data.index.weekday < 5].resample('D').sum().mean()
    return np.array([weekends_avg, business_days_avg]).flatten()

In [ ]:
#| export
def resample_building_data(group:dict)->dict:
    group = group.reset_index(level='ID')
    # Specify columns explicitly for summing
    resampled_group = group.resample('h').agg({'consumption': 'sum'})  # Example if 'consumption' is your numeric column
    resampled_group['ID'] = group['ID'].iloc[0]  # Handle non-numeric separately if needed
    resampled_group = resampled_group.set_index('ID', append=True)
    return resampled_group

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()